In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

seed = 7
np.random.seed(seed)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataset = pd.read_csv('../datasets/pima-indians-diabetes.data.csv', names = names)

In [3]:
dataset.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# split into input (X) and output (Y) variables
X = dataset.values[:,0:8]
Y = dataset.values[:,8]

In [5]:
inputs = len(dataset.columns) - 1

# create model
model = Sequential()
model.add(Dense(12, input_dim = 8, kernel_initializer='random_uniform', activation = 'relu'))
model.add(Dense(8, kernel_initializer='random_uniform' , activation = 'relu'))
model.add(Dense(1, kernel_initializer='random_uniform' , activation = 'sigmoid'))

In [6]:
# compile model
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [7]:
# Fit the model
model.fit(X, Y, epochs=150, batch_size=10)

Train on 768 samples
Epoch 1/150
768/768 [==============================] - 0s 134us/sample - loss: 0.6779 - acc: 0.6510
Epoch 2/150
768/768 [==============================] - 0s 63us/sample - loss: 0.6595 - acc: 0.6497
Epoch 3/150
768/768 [==============================] - 0s 64us/sample - loss: 0.6449 - acc: 0.6549
Epoch 4/150
768/768 [==============================] - 0s 64us/sample - loss: 0.6272 - acc: 0.6602
Epoch 5/150
768/768 [==============================] - 0s 62us/sample - loss: 0.6123 - acc: 0.6823
Epoch 6/150
768/768 [==============================] - 0s 62us/sample - loss: 0.6019 - acc: 0.6940
Epoch 7/150
768/768 [==============================] - 0s 60us/sample - loss: 0.5989 - acc: 0.6823
Epoch 8/150
768/768 [==============================] - 0s 62us/sample - loss: 0.5929 - acc: 0.6823
Epoch 9/150
768/768 [==============================] - 0s 62us/sample - loss: 0.5914 - acc: 0.6797
Epoch 10/150
768/768 [==============================] - 0s 62us/sample - loss: 0.5915 -

In [8]:
# evaluate the model
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 78.65%


## KFold model validation

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy as np

seed = 7
np.random.seed(seed)

In [10]:
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataset = pd.read_csv('../datasets/pima-indians-diabetes.data.csv', names = names)

In [11]:
X = dataset.values[:,0:8]
Y = dataset.values[:,8]

In [12]:
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

cvscores = []

for train, test in kfold.split(X, Y):
    
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim = 8, kernel_initializer = 'random_uniform', activation ='relu'))
    model.add(Dense(8, kernel_initializer = 'random_uniform', activation = 'relu'))
    model.add(Dense(1, kernel_initializer = 'random_uniform', activation = 'sigmoid'))
    
    # Compile model
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam' , metrics = ['acc'])
    
    # Fit the model
    model.fit(X[train], Y[train], epochs=150, batch_size=10, verbose=0)
    
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=0)
    
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

acc: 77.92%
acc: 79.22%
acc: 75.32%
acc: 79.22%
acc: 72.73%
acc: 64.94%
acc: 75.32%
acc: 75.32%
acc: 75.00%
acc: 85.53%
76.05% (+/- 5.01%)


## Evaluate Models with Cross Validation

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

seed = 7
np.random.seed(seed)

In [14]:
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataset = pd.read_csv('../datasets/pima-indians-diabetes.data.csv', names = names)

In [15]:
dataset.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [16]:
X = dataset.values[:,0:8]
Y = dataset.values[:,8]

In [17]:
# Function to create model, required for KerasClassifier
def create_model():
    
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim = 8, kernel_initializer = 'random_uniform', activation ='relu'))
    model.add(Dense(8, kernel_initializer = 'random_uniform', activation = 'relu'))
    model.add(Dense(1, kernel_initializer = 'random_uniform', activation = 'sigmoid'))
    
    # Compile model
    model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
    
    return model

In [18]:
# create model
model = KerasClassifier(build_fn = create_model, epochs=150, batch_size = 10, verbose = 0)

# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = seed)
results = cross_val_score(model, X, Y, cv = kfold)

print(results.mean())

0.7461893260478973


In [19]:
results

array([0.77922076, 0.79220778, 0.76623374, 0.8051948 , 0.67532468,
       0.61038959, 0.77922076, 0.6883117 , 0.7631579 , 0.80263156])

## Grid Search Deep Learning Model Parameters

In [20]:
# MLP for Pima Indians Dataset with grid search via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

seed = 7
np.random.seed(seed)

In [33]:
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataset = pd.read_csv('../datasets/pima-indians-diabetes.data.csv', names = names)

In [34]:
dataset.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [35]:
X = dataset.values[:,0:8]
Y = dataset.values[:,8]

In [32]:
# Function to create model, required for KerasClassifier
def create_model(optimizer = 'rmsprop', init = 'glorot_uniform'):
    
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer = init, activation= 'relu' ))
    model.add(Dense(8, kernel_initializer = init, activation = 'relu' ))
    model.add(Dense(1, kernel_initializer = init, activation = 'sigmoid' ))
    
    # Compile model
    model.compile(loss = 'binary_crossentropy' , optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [38]:
model = KerasClassifier(build_fn=create_model, verbose=0)

# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = np.array([50, 100, 150])
batches = np.array([5, 10, 20])

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size = batches, init = init)

#grid = GridSearchCV(estimator = model, param_grid = param_grid)
#grid_result = grid.fit(X, Y)

In [37]:
# summarize results
#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#for params, mean_score, scores in grid_result.grid_scores_:
#    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

In [42]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

seed = 7
np.random.seed(seed)

In [46]:
# load dataset
dataframe = pd.read_csv('../datasets/iris.csv', header = None)
dataset = dataframe.values

X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

In [50]:
X[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [51]:
Y[:10]

array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa'], dtype=object)

In [52]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [55]:
dummy_y[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [56]:
# define baseline model
def baseline_model():
    
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim = 4, kernel_initializer = 'normal', activation = 'relu' ))
    model.add(Dense(3, kernel_initializer = 'normal', activation = 'sigmoid' ))
    
    # Compile model
    model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['acc'])
    
    return model

In [60]:
estimator = KerasClassifier(build_fn = baseline_model, epochs = 200, batch_size = 5, verbose = 0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)

In [61]:
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 96.67% (4.47%)


## Binary Classification

In [91]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

seed = 7
np.random.seed(seed)

In [92]:
dataframe = pd.read_csv("../datasets/sonar.csv", header=None)
dataset = dataframe.values

dataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [71]:
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [73]:
X[:1]

array([[0.02  , 0.0371, 0.0428, 0.0207, 0.0954, 0.0986, 0.1539, 0.1601,
        0.3109, 0.2111, 0.1609, 0.1582, 0.2238, 0.0645, 0.066 , 0.2273,
        0.31  , 0.2999, 0.5078, 0.4797, 0.5783, 0.5071, 0.4328, 0.555 ,
        0.6711, 0.6415, 0.7104, 0.808 , 0.6791, 0.3857, 0.1307, 0.2604,
        0.5121, 0.7547, 0.8537, 0.8507, 0.6692, 0.6097, 0.4943, 0.2744,
        0.051 , 0.2834, 0.2825, 0.4256, 0.2641, 0.1386, 0.1051, 0.1343,
        0.0383, 0.0324, 0.0232, 0.0027, 0.0065, 0.0159, 0.0072, 0.0167,
        0.018 , 0.0084, 0.009 , 0.0032]])

In [74]:
Y[:10]

array(['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R'], dtype=object)

In [76]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)

encoded_Y = encoder.transform(Y)

In [78]:
encoded_Y[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [79]:
# baseline model
def create_baseline():
    
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim = 60, kernel_initializer = 'normal', activation = 'relu' ))
    model.add(Dense(1, kernel_initializer = 'normal', activation= 'sigmoid' ))
    
    # Compile model
    model.compile(loss = 'binary_crossentropy', optimizer= 'adam' , metrics = ['accuracy'])
    
    return model

In [82]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn = create_baseline, epochs = 100, batch_size = 5, verbose = 0)

kfold = StratifiedKFold(n_splits = 10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, encoded_Y, cv = kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 81.21% (8.52%)


## Model Tunning

In [83]:
estimators = []
estimators.append(('standardize' , StandardScaler()))
estimators.append(('mlp' , KerasClassifier(build_fn = create_baseline, epochs = 100, batch_size = 5, verbose = 0)))
pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = seed)
results = cross_val_score(pipeline, X, encoded_Y, cv = kfold)

print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 85.55% (7.63%)


## Evaluate a Smaller Network

In [85]:
def create_smaller():
    
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim = 60, kernel_initializer = 'normal', activation = 'relu' ))
    model.add(Dense(1, kernel_initializer = 'normal' , activation = 'sigmoid' ))
    
    # Compile model
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [86]:
estimators = []
estimators.append(('standardize' , StandardScaler()))
estimators.append(('mlp' , KerasClassifier(build_fn = create_smaller, epochs = 100, batch_size = 5, verbose = 0)))

pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)

print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 84.60% (7.24%)


## Evaluate a Larger Network

In [89]:
def create_larger():
    
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim = 60, kernel_initializer = 'normal' , activation = 'relu'))
    model.add(Dense(30, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer = 'normal', activation = 'sigmoid'))
    
    # Compile model
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam' , metrics=['accuracy'])
    
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = create_larger, epochs = 100, batch_size = 5, verbose = 0)))

pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = seed)
results = cross_val_score(pipeline, X, encoded_Y, cv = kfold)

print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 85.07% (5.96%)


## Linear regression

In [102]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

seed = 7
np.random.seed(seed)

In [103]:
dataframe = pd.read_csv("../datasets/housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values

X = dataset[:,0:13]
Y = dataset[:,13]

In [104]:
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [105]:
def baseline_model():

    # create model
    #model = Sequential()
    #model.add(Dense(13, input_dim = 13, kernel_initializer = 'normal', activation = 'relu'))
    #model.add(Dense(1, kernel_initializer = 'normal'))
    
    model.add(Dense(13, input_dim = 13, kernel_initializer = 'normal', activation= 'relu'))
    model.add(Dense(6, kernel_initializer = 'normal', activation = 'relu' ))
    model.add(Dense(1, kernel_initializer = 'normal'))
    
    # Compile model
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    
    return model

In [108]:
estimator = KerasRegressor(build_fn = baseline_model, epochs = 100, batch_size = 5, verbose = 0)

kfold = KFold(n_splits = 10, random_state = seed, shuffle = True)
results = cross_val_score(estimator, X, Y, cv=kfold)

print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Baseline: -28.63 (13.57) MSE


In [109]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn = baseline_model, epochs = 50, batch_size = 5, verbose = 0)))

pipeline = Pipeline(estimators)

kfold = KFold(n_splits = 10, random_state = seed, shuffle = True)
results = cross_val_score(pipeline, X, Y, cv=kfold)

print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -19.15 (6.46) MSE
